# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 12:09PM,254660,10,Enova-11181,Emerginnova,Released
1,Jan 10 2023 12:06PM,254659,10,9011276,"Snap Medical Industries, LLC",Released
2,Jan 10 2023 12:06PM,254659,10,9011277,"Snap Medical Industries, LLC",Released
3,Jan 10 2023 12:06PM,254659,10,9011278,"Snap Medical Industries, LLC",Released
4,Jan 10 2023 12:06PM,254659,10,9011279,"Snap Medical Industries, LLC",Released
5,Jan 10 2023 12:06PM,254659,10,9011280,"Snap Medical Industries, LLC",Released
6,Jan 10 2023 12:06PM,254659,10,SNAP11023,"Snap Medical Industries, LLC",Released
7,Jan 10 2023 12:06PM,254659,10,SNAP11023B,"Snap Medical Industries, LLC",Released
8,Jan 10 2023 12:06PM,254659,10,SNAP1199530,"Snap Medical Industries, LLC",Released
9,Jan 10 2023 12:06PM,254659,10,SNAPnet32-34043987,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,254656,Released,11
26,254657,Released,1
27,254658,Released,1
28,254659,Released,9
29,254660,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254656,NaN,NaN,11.0
254657,NaN,NaN,1.0
254658,NaN,NaN,1.0
254659,NaN,NaN,9.0
254660,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254598,0.0,0.0,1.0
254599,0.0,0.0,1.0
254607,0.0,6.0,3.0
254608,0.0,5.0,11.0
254609,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254598,0,0,1
254599,0,0,1
254607,0,6,3
254608,0,5,11
254609,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254598,0,0,1
1,254599,0,0,1
2,254607,0,6,3
3,254608,0,5,11
4,254609,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254598,,,1
1,254599,,,1
2,254607,,6,3
3,254608,,5,11
4,254609,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 12:09PM,254660,10,Emerginnova
1,Jan 10 2023 12:06PM,254659,10,"Snap Medical Industries, LLC"
10,Jan 10 2023 12:02PM,254658,10,Yusen – 3D Matrix
11,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc."
22,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc."
23,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc."
24,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation
25,Jan 10 2023 11:16AM,254652,10,MedStone Pharma LLC
32,Jan 10 2023 11:14AM,254651,10,"Senseonics, Incorporated"
33,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 12:09PM,254660,10,Emerginnova,,,1
1,Jan 10 2023 12:06PM,254659,10,"Snap Medical Industries, LLC",,,9
2,Jan 10 2023 12:02PM,254658,10,Yusen – 3D Matrix,,,1
3,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",,,11
4,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",,,1
5,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",,,1
6,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,,,1
7,Jan 10 2023 11:16AM,254652,10,MedStone Pharma LLC,,2,5
8,Jan 10 2023 11:14AM,254651,10,"Senseonics, Incorporated",,,1
9,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 12:09PM,254660,10,Emerginnova,1,,
1,Jan 10 2023 12:06PM,254659,10,"Snap Medical Industries, LLC",9,,
2,Jan 10 2023 12:02PM,254658,10,Yusen – 3D Matrix,1,,
3,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11,,
4,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1,,
5,Jan 10 2023 11:40AM,254655,21,"NBTY Global, Inc.",1,,
6,Jan 10 2023 11:36AM,254654,10,ISDIN Corporation,1,,
7,Jan 10 2023 11:16AM,254652,10,MedStone Pharma LLC,5,2,
8,Jan 10 2023 11:14AM,254651,10,"Senseonics, Incorporated",1,,
9,Jan 10 2023 10:52AM,254650,15,"Brookfield Pharmaceuticals, LLC",18,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 12:09PM,254660,10,Emerginnova,1,,
1,Jan 10 2023 12:06PM,254659,10,"Snap Medical Industries, LLC",9,,
2,Jan 10 2023 12:02PM,254658,10,Yusen – 3D Matrix,1,,
3,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11,,
4,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 12:09PM,254660,10,Emerginnova,1.0,NaN,NaN
1,Jan 10 2023 12:06PM,254659,10,"Snap Medical Industries, LLC",9.0,NaN,NaN
2,Jan 10 2023 12:02PM,254658,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11.0,NaN,NaN
4,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 12:09PM,254660,10,Emerginnova,1.0,0.0,0.0
1,Jan 10 2023 12:06PM,254659,10,"Snap Medical Industries, LLC",9.0,0.0,0.0
2,Jan 10 2023 12:02PM,254658,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Jan 10 2023 11:52AM,254656,15,"Alliance Pharma, Inc.",11.0,0.0,0.0
4,Jan 10 2023 11:51AM,254657,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2546486,27.0,4.0,7.0
15,1527786,60.0,11.0,0.0
19,1018492,40.0,3.0,2.0
21,1018509,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2546486,27.0,4.0,7.0
1,15,1527786,60.0,11.0,0.0
2,19,1018492,40.0,3.0,2.0
3,21,1018509,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,27.0,4.0,7.0
1,15,60.0,11.0,0.0
2,19,40.0,3.0,2.0
3,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,27.0
1,15,Released,60.0
2,19,Released,40.0
3,21,Released,4.0
4,10,Executing,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21
Status,,,,
Completed,7.0,0.0,2.0,0.0
Executing,4.0,11.0,3.0,0.0
Released,27.0,60.0,40.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21
0,Completed,7.0,0.0,2.0,0.0
1,Executing,4.0,11.0,3.0,0.0
2,Released,27.0,60.0,40.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21
0,Completed,7.0,0.0,2.0,0.0
1,Executing,4.0,11.0,3.0,0.0
2,Released,27.0,60.0,40.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()